# DM-26087 Array multiplication timing and floating point significance


In this notebook, we do small empirical tests to determine which way is it best to write array multiplication expressions for calculation speed in numpy.

 * Python seems to evaulate multiplications left-to right
 * ``Array*number`` multiplications goes through all items, therefore if there are multiple factors they should be multiplied together first for speed. This is achieved if the numerical factors come first, otherwise use parentheses.
 * ``int`` array operations is slower! FPU?
 
 --------
 
 * tiny is the smallest _usable_ number i.e. that has all the digits and the smallest exponent, but smaller numbers are not immediately zero while the significand remains any useful bits. ``tiny*epsilon`` still have ``~1`` digit left, ``tiny*epsilon*0.5 == 0`` however.

* Of course, ``1.+0.5 * eps == 1.``

In [2]:
import numpy as np

In [2]:
A = np.arange(1024*1024,dtype=float).reshape(1024,1024)

In [3]:
%timeit 3*4*5*A

611 µs ± 16.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [4]:
%timeit (3*4*5)*A

618 µs ± 23.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [5]:
%timeit A*3*4*5

1.47 ms ± 17.4 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [6]:
%timeit A*(3*4*5)

616 µs ± 22.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


Now repeat with int array

In [7]:
A = np.arange(1024*1024,dtype=int).reshape(1024,1024)

In [8]:
%timeit 3*4*5*A

640 µs ± 16.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [9]:
%timeit (3*4*5)*A

649 µs ± 21.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [10]:
%timeit A*3*4*5

1.49 ms ± 22.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [11]:
%timeit A*(3*4*5)

646 µs ± 25.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


--------

### Demonstration of the key behavior of the floating point limit values

In [3]:
tiny = np.finfo(float).tiny
eps = np.finfo(float).eps

In [4]:
print(1.+0.5*eps)
print(tiny*0.5)
print(tiny*eps)
print(tiny*eps*0.5)

1.0
1.1125369292536007e-308
5e-324
0.0
